## Primeiros passos com Keras

Esta contribuição demonstra a facilidade do desenvolvimento de uma rede neural convolucional LeNet-5 utilizando framework keras com TensorFlow em background e utilizando o dataset Cifar10.
 

In [178]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.callbacks import TensorBoard
import os
import warnings
from time import time
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

warnings.simplefilter("ignore")

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 1303535744627528729
]


##### Carrega dataset Cifar10 e faz o split em treino e test.

In [157]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


##### Parâmetros da rede e de treinamento

In [158]:
batch_size = 32
num_classes = 10
epochs = 100
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'Keras_Cifar10_Model'
INPUT_SHAPE = x_train.shape[1:]

In [159]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

##### Criação da rede convolucional

In [160]:
model = Sequential()
model.add(Conv2D(32,(3,3), padding='same', input_shape=INPUT_SHAPE))
model.add(Activation('elu'))
model.add(Conv2D(32,(3,3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.25))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

##### Adicionando função de otimização

In [161]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

##### Treinando a rede

In [162]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

##### Chamada do TensorBoard

In [163]:
tensorboard = TensorBoard(log_dir="logs/{}".format(time()))
keras.callbacks.TensorBoard(log_dir='./logs',
                            histogram_freq=0, 
                            batch_size=32, 
                            write_graph=True, 
                            write_grads=False, 
                            write_images=False, 
                            embeddings_freq=0, 
                            embeddings_layer_names=None, 
                            embeddings_metadata=None, 
                            embeddings_data=None, 
                            update_freq='epoch')

In [164]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [165]:
if not data_augmentation:
    print('Not using data augmentation')
    model.fit(x_train, y_train,
             batch_size=batch_size,
             epochs=epochs,
             validation_data=(y_test,x_test),
             shuffle=True)
else:
    print('Using-real-time data augmentation')
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        zca_epsilon=1e-6,
        rotation_range=0,
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0.,
        zoom_range=0.,
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0) 

Using-real-time data augmentation


In [166]:
datagen.fit(x_train)

In [167]:
model.fit_generator(datagen.flow(x_train, y_train,
                                 batch_size=batch_size),
                    epochs=epochs,
                    validation_data=(x_test,y_test),
                    workers=4,
                    steps_per_epoch=10,
                    callbacks=[tensorboard])

Epoch 1/100
10/10 [==============================] - 18s 2s/step - loss: 2.3976 - acc: 0.1156 - val_loss: 2.2297 - val_acc: 0.1848
Epoch 2/100
10/10 [==============================] - 19s 2s/step - loss: 2.2608 - acc: 0.1687 - val_loss: 2.1783 - val_acc: 0.2024
Epoch 3/100
10/10 [==============================] - 22s 2s/step - loss: 2.1694 - acc: 0.1750 - val_loss: 2.1322 - val_acc: 0.2034
Epoch 4/100
10/10 [==============================] - 18s 2s/step - loss: 2.1417 - acc: 0.2062 - val_loss: 2.0727 - val_acc: 0.2654
Epoch 5/100
10/10 [==============================] - 17s 2s/step - loss: 2.0637 - acc: 0.2531 - val_loss: 2.0534 - val_acc: 0.2559
Epoch 6/100
10/10 [==============================] - 17s 2s/step - loss: 2.0783 - acc: 0.2219 - val_loss: 2.0148 - val_acc: 0.2759
Epoch 7/100
10/10 [==============================] - 18s 2s/step - loss: 2.1002 - acc: 0.2563 - val_loss: 1.9974 - val_acc: 0.2720
Epoch 8/100
10/10 [==============================] - 16s 2s/step - loss: 2.0225 - a

##### Salvando Modelo

In [168]:
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)

model_path = os.path.join(save_dir,model_name)
model.save(model_path)
print('Modelo treinado e salvo em %s ' % model_path)

Modelo treinado e salvo em C:\Users\leandro.r.a.silvA\MachineLearning-Leandro\CNN\saved_models\Keras_Cifar10_Model 


##### Perda e Acurácia do modelo aplicado aos dados de teste

In [169]:
score = model.evaluate(x_test, y_test, verbose=1)
print('Perda em teste:', score[0])
print('Acurácia em teste:', score[1])

10000/10000 [==============================] - 15s 2ms/step
Perda em teste: 1.585744954109192
Acurácia em teste: 0.441


##### Chamada do Tensorflow

Executar o comando abaixo no shell do anaconda e copiar o link que será gerado, composto pelo nome do host e porta 6006.

In [ ]:
tensorboard --logs=/C:\Users\leandro.r.a.silvA\MachineLearning-Leandro\CNN\logs\1540558495.3674333